In [1]:
from xml.sax import handler
import streamlit as st
import tensorflow as tf
import numpy as np
import json
import pandas as pd
from io import BytesIO
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

2022-04-11 21:31:51.848 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
with open("./prediction_dict.json", "r") as file:
    dictionary = json.load(file)

In [3]:
path = './images_by_class/validation/class_frasiers_dolphin/4d817d2f3e6298.jpg'
image = np.array(tf.keras.utils.load_img(
    path,
    grayscale=False,
    color_mode='rgb',
    target_size=(224,224),
    interpolation='bilinear'
))
#image = tf.io.decode_image(bytes_data, channels=3).numpy()
#Now we have a numpy array to process

#desired_input_shape = (224, 224)
#image = tf.keras.preprocessing.image.smart_resize( image, desired_input_shape, interpolation='bilinear')
print(type(image))
print(image.shape)
#expanded_img_array = np.expand_dims(img_array, axis=0)
#preprocessed_img = preprocess_input(expanded_img_array)

#print(image[0])
image = image/255
#print(image[0])
image = np.expand_dims(image, axis=0)
print(image.shape)

<class 'numpy.ndarray'>
(224, 224, 3)
(1, 224, 224, 3)


In [4]:
with CustomObjectScope(
    {'GlorotUniform': glorot_uniform()}):
    model = load_model('./models/mobilenet_transfer_model.h5')


preds = model.predict(image)
pred = dictionary[str(np.argmax(preds))]
pred = ' '.join(pred.split('_')[1:])
print(f"This is a picture of a {pred}!")

2022-04-11 21:31:57.070719: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


This is a picture of a dusky dolphin!


In [5]:
dictionary['8']

'class_dusky_dolphin'

In [16]:
preds = list(preds[0])
preds

[0.0003867156,
 7.768215e-06,
 0.011837087,
 0.09882202,
 4.4075186e-06,
 0.00033812542,
 0.02404251,
 0.0003336545,
 0.4138474,
 0.016664138,
 0.00011476831,
 0.006878262,
 0.0035026819,
 5.8372185e-05,
 0.0041753775,
 0.00010678837,
 0.00045687373,
 0.00027642326,
 0.14976625,
 6.297103e-05,
 0.007866501,
 0.0013878342,
 0.03758607,
 0.03505474,
 0.00033713417,
 0.0010953698,
 6.9397097e-06,
 0.18305664,
 0.0018866031,
 3.9543855e-05]

In [18]:
data =[]
for index, pred in enumerate(preds):
    categ = dictionary[str(index)]
    categ = ' '.join(categ.split('_')[1:])
    data.append([categ,pred])
data

[['beluga', 0.0003867156],
 ['blue whale', 7.768215e-06],
 ['bottlenose dolphin', 0.011837087],
 ['bottlenose dolpin', 0.09882202],
 ['brydes whale', 4.4075186e-06],
 ['commersons dolphin', 0.00033812542],
 ['common dolphin', 0.02404251],
 ['cuviers beaked whale', 0.0003336545],
 ['dusky dolphin', 0.4138474],
 ['false killer whale', 0.016664138],
 ['fin whale', 0.00011476831],
 ['frasiers dolphin', 0.006878262],
 ['globis', 0.0035026819],
 ['gray whale', 5.8372185e-05],
 ['humpback whale', 0.0041753775],
 ['kiler whale', 0.00010678837],
 ['killer whale', 0.00045687373],
 ['long finned pilot whale', 0.00027642326],
 ['melon headed whale', 0.14976625],
 ['minke whale', 6.297103e-05],
 ['pantropic spotted dolphin', 0.007866501],
 ['pilot whale', 0.0013878342],
 ['pygmy killer whale', 0.03758607],
 ['rough toothed dolphin', 0.03505474],
 ['sei whale', 0.00033713417],
 ['short finned pilot whale', 0.0010953698],
 ['southern right whale', 6.9397097e-06],
 ['spinner dolphin', 0.18305664],
 ['

In [91]:
df = pd.DataFrame(data).rename(columns={0: "Species Class", 1: "Probability %"})
df = df.sort_values(by=['Probability %'], ascending = False).head(15)

In [90]:
import altair as alt
bars = alt.Chart(df).mark_bar().encode(
    y=alt.Y('Species Class:N', sort='-x'),
    x="Probability %:Q",
    color=alt.Color('Species Class', scale=alt.Scale(scheme='blues'), legend = None)
    
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Probability %:Q'
)

chart = (bars + text).properties(height=900, title = 'Class Probabilities by Percentage')
chart

alt.LayerChart(...)